# 因果语言模型训练实例(模型续写，就是CPT)

## Step1 导入相关包

In [1]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer, BloomForCausalLM

## Step2 加载数据集

In [2]:
ds = Dataset.load_from_disk('./data/wiki_cn_filtered')
ds[0]

{'source': 'wikipedia.zh2307',
 'completion': "西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆，馆长是锺明善。\n历史\n2004年9月20日开始筹建，2013年4月8日正式建成开馆，位于西安交通大学兴庆校区陕西省西安市咸宁西路28号。建筑面积6,800平米，展厅面积4,500平米，馆藏文物4,900余件。包括历代艺术文物馆、碑石书法馆、西部农民画馆、邢良坤陶瓷艺术馆、陕西秦腔博物馆和书画展厅共五馆一厅。\n营业时间\n* 周一至周六：上午九点至十二点，下午一点至五点\n* 周日闭馆"}

## Step3 数据集处理

In [ ]:
# 就是所有的句子增加一个EOS的停止符

tokenizer = AutoTokenizer.from_pretrained('Langboat/bloom-389m-zh')

def process_function(examples):
    contents = [e + tokenizer.eos_token for e in examples['completion']]
    return tokenizer(contents, max_length=384, truncation=True)

In [ ]:
tokenizer_datasets = ds.map(process_function, batched=True, remove_columns=ds.column_names)

## Step4 创建模型

In [ ]:
model = AutoModelForCausalLM.from_pretrained('Langboat/bloom-389m-zh')

## Step5 配置训练参数

In [ ]:
args = TrainingArguments(
    output_dir='./causual_llm',
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=50,
    num_train_epochs=1,
    fp16=True
)

## Step6 创建训练器

In [ ]:
trainer = Trainer(
    model = model,
    args=args,
    train_dataset=tokenizer_datasets,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)

## Step7 模型训练

In [ ]:
trainer.train()

## Step8 模型推理

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

In [ ]:
pipe("西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安", max_length=128, do_sample=True)